In [ ]:
# I have seperate Q Tables for Black and White players

In [29]:
from sys import maxsize
import time
import json
import numpy as np
import pandas as pd

N=5
komi = N/2
WIN_REWARD = 1.0
gamma = 0.9
alpha = 0.7
LOSS_REWARD = 0.0
WIN = False
INITIAL = 0.5

blackPath = r"C:\Users\abc\Desktop\USC\Spring 20\CSCI 561 - AI\HW\HW2\q_values_black.json"
whitePath = r"C:\Users\abc\Desktop\USC\Spring 20\CSCI 561 - AI\HW\HW2\q_values_white.json"

In [30]:
df = pd.read_csv(r"C:\Users\abc\Desktop\USC\Spring 20\CSCI 561 - AI\HW\HW2\Q history\history_1000_ab-random.csv")
with open(blackPath) as infile:
    q_black = json.load(infile)
with open(whitePath) as infile:
    q_white = json.load(infile)

In [16]:
bad_examples=[]
train = []
df2 = pd.DataFrame()
for row in range(df.shape[0]):
    curr = df.iloc[row]
    if row==df.shape[0]-1:
        nex = None
    else:
        nex = df.iloc[row+1]
    df2 = df2.append(curr)
    end = False
    if (type(nex==None)==bool) or (curr.iam!=nex.iam):
        end=True
        if len(df2)>12:
            bad_examples.append(len(train))
        train.append(df2)
        df2=pd.DataFrame()

In [17]:
def createIntMatrix(string):
    state=[]
    row=[]
    count=0
    for char in string:
        try:
            i = int(char)
            row.append(i)
            count+=1
            if count==5:
                state.append(row)
                row,count = [],0
        except:
            continue
    return state

def action(state, player,rival):
    vacancies = []
    for row in range(0,len(state)):
        for val in range(0,len(state[row])):
            if state[row][val]==0:
#                 if (row,val)==KO:
#                     continue
                flag=False
                temp = copyState(state)
                temp[row][val]=player
                temp2 = checkLiberty(copyState(temp),rival)
                temp3 = checkLiberty(copyState(temp),player)
                opp1,opp2,me1,me2=0,0,0,0
                for u in temp:
                    opp1+=u.count(rival)
                    me1+=u.count(player)
                for v in temp2:
                    opp2+=v.count(rival)
                for w in temp3:
                    me2+=w.count(player)
                if opp2<opp1 or me2==me1:
                    vacancies.append((row,val))
    return vacancies

def checkLiberty(state, whose):
    #liberty elimination
    if type(state)==str:
        state=createIntMatrix(state)
    for row in range(0,len(state)):
        for val in range(0,len(state[row])):
            curr = state[row][val]
            if curr==whose:
                liberty = [(row,val)]
                seen = set({})
                x = [1,-1,0,0]
                y = [0,0,-1,1]
                flag = True
                while len(liberty)!=0:
                    rt,vt = liberty.pop(0)
                    seen.add((rt,vt))
                    for i in range(0,len(x)):
                        r = rt+x[i]
                        v = vt+y[i]
                        if (r<0) or (v<0) or r>=len(state) or v>=len(state[row]):
                            continue
                        if state[r][v]==0:
                            flag = False
                            break
                        elif state[r][v]==curr and ((r,v) not in seen) and ((r,v) not in liberty):
                            liberty.append((r,v))
                    if not flag:
                        break
                if flag:
                    for r,v in seen:
                        state[r][v] = 0
    return state

def copyState(state):
    newState = []
    for row in state:
        newState.append(row[:])
    return newState

def evaluate(state):
    white = komi
    black = 0
    for row in state:
        white+=row.count(2)
        black+=row.count(1)
    return white-black

In [23]:
def learn(df,result,q_values):
    state = df.iloc[-1].state
    iam = int(df.iloc[-1].iam)
    opp = 1 if iam==2 else 2

    if result:
        reward = WIN_REWARD
    else:
        reward = LOSS_REWARD

    max_q_value = -float('inf')
    for row in range(len(df)-1,-1,-1):
        curr = df.iloc[row]
        state, move = curr.state, (int(curr.move[1]),int(curr.move[4]))
        if str(state) not in q_values.keys():
            q_values[str(state)] = np.full((5, 5),fill_value=-np.inf)
            for i,j in action(createIntMatrix(state),iam,opp):
                q_values[str(state)][i][j] = INITIAL
        if max_q_value == - float('inf'):
            q_values[str(state)][move[0]][move[1]] = reward
        else:
            q_values[str(state)][move[0]][move[1]] = q_values[str(state)][move[0]][move[1]] * (1 - alpha) + alpha * gamma * max_q_value
        max_q_value = np.max(q_values[str(state)])
        try:
            q_values[str(state)] = q_values[str(state)].tolist()
        except:
            continue
    return q_values

In [24]:
counter = 0
won1,won2=0,0
for data in train:
    counter+=1
    iam = int(data['iam'].values[0])
    print('Training for player - ',iam,' and game number - ',counter)
    opp = 1 if iam==2 else 2
    lastState = createIntMatrix(data['state'].values[-1])
    i,j = int(data['move'].values[-1][1]),int(data['move'].values[-1][4])
    result = True
    
    #Checking if less length due to invalid move by agent
    if data.shape[0]<12:
        if lastState[i][j]!=0 or ((i,j) not in action(lastState, iam, opp)):
            continue
    
    #make my player's last move
    lastState[i][j]=iam
    lastState=checkLiberty(copyState(lastState),opp)
    #Checking if opponent's (white) last move defeats me or not
    if iam==1:
        actions = action(lastState,opp,iam)
        best_move = tuple((-float('inf'),None))
        for a in actions:
            u,v = a
            s = copyState(lastState)
            s[u][v]=2
            s = checkLiberty(copyState(s),iam)
            if evaluate(s)>0:
                result = False
                break
    elif evaluate(lastState)<0:
        result = False
    if iam==1:
        won1+=result
        q_black = learn(data,result,q_black)
    else:
        won2+=result
        q_white = learn(data,result,q_white)

Training for player -  2  and game number -  1
Training for player -  1  and game number -  2
Training for player -  2  and game number -  3
Training for player -  1  and game number -  4
Training for player -  2  and game number -  5
Training for player -  1  and game number -  6
Training for player -  2  and game number -  7
Training for player -  1  and game number -  8
Training for player -  2  and game number -  9
Training for player -  1  and game number -  10
Training for player -  2  and game number -  11
Training for player -  1  and game number -  12
Training for player -  2  and game number -  13
Training for player -  1  and game number -  14
Training for player -  2  and game number -  15
Training for player -  1  and game number -  16
Training for player -  2  and game number -  17
Training for player -  1  and game number -  18
Training for player -  2  and game number -  19
Training for player -  1  and game number -  20
Training for player -  2  and game number -  21
T

Training for player -  2  and game number -  185
Training for player -  1  and game number -  186
Training for player -  2  and game number -  187
Training for player -  1  and game number -  188
Training for player -  2  and game number -  189
Training for player -  1  and game number -  190
Training for player -  2  and game number -  191
Training for player -  1  and game number -  192
Training for player -  2  and game number -  193
Training for player -  1  and game number -  194
Training for player -  2  and game number -  195
Training for player -  1  and game number -  196
Training for player -  2  and game number -  197
Training for player -  1  and game number -  198
Training for player -  2  and game number -  199
Training for player -  1  and game number -  200
Training for player -  2  and game number -  201
Training for player -  1  and game number -  202
Training for player -  2  and game number -  203
Training for player -  1  and game number -  204
Training for player 

Training for player -  2  and game number -  399
Training for player -  1  and game number -  400
Training for player -  2  and game number -  401
Training for player -  1  and game number -  402
Training for player -  2  and game number -  403
Training for player -  1  and game number -  404
Training for player -  2  and game number -  405
Training for player -  1  and game number -  406
Training for player -  2  and game number -  407
Training for player -  1  and game number -  408
Training for player -  2  and game number -  409
Training for player -  1  and game number -  410
Training for player -  2  and game number -  411
Training for player -  1  and game number -  412
Training for player -  2  and game number -  413
Training for player -  1  and game number -  414
Training for player -  2  and game number -  415
Training for player -  1  and game number -  416
Training for player -  2  and game number -  417
Training for player -  1  and game number -  418
Training for player 

Training for player -  1  and game number -  592
Training for player -  2  and game number -  593
Training for player -  1  and game number -  594
Training for player -  2  and game number -  595
Training for player -  1  and game number -  596
Training for player -  2  and game number -  597
Training for player -  1  and game number -  598
Training for player -  2  and game number -  599
Training for player -  1  and game number -  600
Training for player -  2  and game number -  601
Training for player -  1  and game number -  602
Training for player -  2  and game number -  603
Training for player -  1  and game number -  604
Training for player -  2  and game number -  605
Training for player -  1  and game number -  606
Training for player -  2  and game number -  607
Training for player -  1  and game number -  608
Training for player -  2  and game number -  609
Training for player -  1  and game number -  610
Training for player -  2  and game number -  611
Training for player 

Training for player -  1  and game number -  808
Training for player -  2  and game number -  809
Training for player -  1  and game number -  810
Training for player -  2  and game number -  811
Training for player -  1  and game number -  812
Training for player -  2  and game number -  813
Training for player -  1  and game number -  814
Training for player -  2  and game number -  815
Training for player -  1  and game number -  816
Training for player -  2  and game number -  817
Training for player -  1  and game number -  818
Training for player -  2  and game number -  819
Training for player -  1  and game number -  820
Training for player -  2  and game number -  821
Training for player -  1  and game number -  822
Training for player -  2  and game number -  823
Training for player -  1  and game number -  824
Training for player -  2  and game number -  825
Training for player -  1  and game number -  826
Training for player -  2  and game number -  827
Training for player 

In [27]:
# won1,won2, len(q_black), len(q_white)
q_black

{'[[2, 0, 0, 0, 0], [2, 2, 1, 1, 2], [2, 2, 1, 2, 1], [2, 1, 1, 0, 1], [2, 1, 1, 1, 1]]': [[-inf,
   0.5,
   0.5,
   0.5,
   1.0],
  [-inf, -inf, -inf, -inf, -inf],
  [-inf, -inf, -inf, -inf, -inf],
  [-inf, -inf, -inf, 0.5, -inf],
  [-inf, -inf, -inf, -inf, -inf]],
 '[[0, 0, 0, 0, 0], [2, 2, 1, 1, 2], [2, 2, 1, 2, 1], [2, 1, 1, 0, 1], [2, 0, 1, 1, 1]]': [[0.5,
   0.5,
   0.5,
   0.5,
   0.5],
  [-inf, -inf, -inf, -inf, -inf],
  [-inf, -inf, -inf, -inf, -inf],
  [-inf, -inf, -inf, 0.5, -inf],
  [-inf, 0.8640000000000001, -inf, -inf, -inf]],
 '[[0, 0, 0, 0, 0], [2, 2, 1, 0, 2], [2, 2, 1, 2, 1], [2, 1, 1, 2, 1], [2, 0, 1, 1, 1]]': [[0.5,
   0.5,
   0.5,
   0.5,
   0.5],
  [-inf, -inf, -inf, 0.7367400000000001, -inf],
  [-inf, -inf, -inf, -inf, -inf],
  [-inf, -inf, -inf, -inf, -inf],
  [-inf, 0.5, -inf, -inf, -inf]],
 '[[0, 0, 0, 0, 0], [2, 2, 1, 0, 2], [2, 2, 0, 2, 1], [2, 1, 1, 2, 1], [0, 0, 1, 1, 1]]': [[0.5,
   0.5,
   0.5,
   0.5,
   0.5],
  [-inf, -inf, -inf, 0.5, -inf],
  [-inf, -

In [28]:
with open(blackPath, 'w') as outfile:
    json.dump(q_black, outfile)
with open(whitePath, 'w') as outfile:
    json.dump(q_white, outfile)